<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網路爬蟲與多執行緒練習



In [ ]:
#@title UU看書網
#@markdown 還在修正的程式，可以直接從這一個區塊執行

def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  reg=requests.get(art_url)
  reg.encoding="utf-8"
  soup=BeautifulSoup(reg.text)
  content=soup.find(name="div",id='contentbox')
  print(art_id)
  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"

  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://tw.uukanshu.com/b/239755/" #@param {type:'string'}
title="\u6211\u5BB6\u5A18\u5B50\uFF0C\u4E0D\u5C0D\u52C1" #@param {type:"string"}
author="\u4E00\u87EC\u77E5\u590F" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
# soup=BeautifulSoup(reg.text,"html.parser")
reg.encoding="utf-8"
soup=BeautifulSoup(reg.text)
articles=soup.find(name="ul",id="chapterList").find_all("a")

links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  links.append([i.get("title"),f"{sites}{href}"])
# links.sort(key=lambda x: x[1])
files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]

# 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)
for link in links:
  get_html(link)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text[::-1] ]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title UU看書網
#@markdown 還在修正的程式，可以直接從這一個區塊執行

def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
  chk_cont=False
except:
  print("目錄已存在")
  chk_cont=True
os.chdir("/content/tmp")
# os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  reg=requests.get(art_url)
  reg.encoding="utf-8"
  soup=BeautifulSoup(reg.text)
  # content="<br>\n".join([tag.string for tag in soup.find(name="div",id='nr').find_all("p")])
  tmp=soup.find(name="div",id='nr').find_all("p")
  print(art_id)
  # pages=soup.find('h1').getText()
  # numbers = re.findall(r'\d+', pages)
  check=soup.find("div",class_="operate").find_all("a")
  if (check[-1].getText()=="下一頁"):
    page_url=sites+check[-1].get("href")
  # if (len(numbers) >1):
  #   last_number=numbers[-1]
  #   # print(last_number)
  #   for i in range(2,int(last_number)+1):
  #     page_url=art_url[:-5]+f"_{i}.html"
    content_extend=get_html_page(page_url)
    tmp=tmp+content_extend


  content="\n".join([str(itm) for itm in tmp])
  # print("<br>".join([tag.string for tag in content]))
  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"

  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


def get_html_page(page_url):
  reg=requests.get(page_url)

  reg.encoding="utf-8"
  soup=BeautifulSoup(reg.text)
  tmp1=soup.find(name="div",id='nr').find_all("p")
  check=soup.find("div",class_="operate").find_all("a")
  if (check[-1].getText()=="下一頁"):
    page_url=sites+check[-1].get("href")
  # if (len(numbers) >1):
  #   last_number=numbers[-1]
  #   # print(last_number)
  #   for i in range(2,int(last_number)+1):
  #     page_url=art_url[:-5]+f"_{i}.html"
    content_extend=get_html_page(page_url)
    tmp1=tmp1+content_extend

  return tmp1



#@markdown 書籍目錄網址
url="https://www.uuread.tw/28100" #@param {type:'string'}
title="系統賦我長生，我熬死了所有人" #@param {type:"string"}
author="一隻榴蓮3號" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
# soup=BeautifulSoup(reg.text,"html.parser")
reg.encoding="utf-8"
soup=BeautifulSoup(reg.text)
articles=soup.find(name="ul",id="newlist").find_all("a")

links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  links.append([i.get("title"),f"{sites}{href}"])
# links.sort(key=lambda x: x[1])
files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]

# # 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)

if chk_cont:
  start_num=int(len(os.listdir())-1)
  print(start_num)
else:
  start_num=0
for index,link in enumerate(links[start_num:],start_num+1):
  print(index)
  get_html(link)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text ]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title UU看書
#@markdown 還在修正的程式，可以直接從這一個區塊執行

def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  reg=requests.get(art_url)
  reg.encoding="utf-8"
  soup=BeautifulSoup(reg.text)
  content=soup.find(name="div",id='TextContent')
  print(art_id)
  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://www.uuks5.com/book/632203/" #@param {type:'string'}
title="豪門梟士" #@param {type:"string"}
author="雲山風海" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
soup=BeautifulSoup(reg.text,"html.parser")
reg.encoding="utf-8"


articles=soup.find(name="ul",id="chapterList").find_all(name="a")

links=[]
# # len(articles)
for i in articles:
  href=i.get("href")
  # print(i.text)
  links.append([i.text,f"{sites}{href}"])
# links.sort(key=lambda x: x[1])
files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]


for link in links:
  get_html(link)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title 飄天文學
#@markdown 還在修正的程式，可以直接從這一個區塊執行
def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  # print(art_url)
  req=requests.get(art_url)
  req.encoding="gbk"
  soup=BeautifulSoup(req.text)
  # content=soup.find('div', {'id': 'content', 'class': 'fonts_mesne'})

  print(art_id)


  # Find all the siblings of the table element up to the center element
  content = []
  content=soup.body
  for tag in content.find_all(['div', 'h1','table','script','center']):
    tag.extract()

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)



  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://www.ptwxz.co/ptjtoio" #@param {type:'string'}
title="系統賦我長生，我熬死了所有人" #@param {type:"string"}
author="一隻榴蓮3號" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url
# sites=url[:url.rfind("/")]

reg=requests.get(url)
reg.encoding = 'gbk'
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)

articles=soup.find(name="div",class_="centent").find_all("a")

links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  links.append([chinese.s2t(i.text),f"{sites}/{href}"])
files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]

# 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)
for link in links:
  get_html(link)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title 69書吧
#@markdown 還在修正的程式，可以直接從這一個區塊執行
def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:]
  # print(art_url)
  req=requests.get(art_url)
  req.encoding="gbk"
  soup=BeautifulSoup(req.text)
  print(art_id)

  # Find all the siblings of the table element up to the center element
  content = []
  content=soup.find(name="div",class_="txtnav")

  for tag in content.find_all(['div','h1','script','center']):
    tag.extract()


  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  # print(text)

  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://www.69shuba.com/book/27232/" #@param {type:'string'}
title="\u5929\u9053\u5716\u66F8\u9928" #@param {type:"string"}
author="\u6A6B\u6383\u5929\u6DAF" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url
# sites=url[:url.rfind("/")]

reg=requests.get(url)
reg.encoding = 'gbk'
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)

articles=soup.find(name="div",id="catalog").find_all("a")

links=[]
# # len(articles)
for i in articles:
  href=i.get("href")
  links.append([chinese.s2t(i.text),href])

files_text=[link[1][link[1].rfind("/")+1:]+".txt" for link in links]

# 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)
for link in links:
  get_html(link)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))

In [ ]:
#@title 笔趣阁(bqg9527.net/)
#@markdown 還在修正的程式，可以直接從這一個區塊執行

def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese


try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]

  soup=BeautifulSoup(requests.get(art_url).text)
  # content=soup.find(name="div",id='nr1')
  content=soup.find(name="div",id="content")
  print(art_id)
  # print(soup)

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  # print(text)


  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://www.bqg9527.net/book/342930/" #@param {type:'string'}
title="我出錢你出命，我倆一起神經病" #@param {type:"string"}
author="我煞費苦心" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

# sites=url[:url.find("/",8)+1]
# sites=url[:url.rfind("/")]
sites=url
reg=requests.get(url)
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)
# articles=soup.find_all(name="ul",class_="chapter")[1].find_all("a")
articles=soup.find(name="div",id="list").find_all("dt")[1].find_next_siblings('dd')

# second_dt=soup.find_all('dt')[1]
# dd_tags = second_dt.find_next_siblings('dd')

links=[]
# len(articles)
for itm in articles:
  i=itm.find("a")
  href=i.get("href")
  if href[-4:]!= "html":
    continue
  links.append([i.text,f"{sites}{href}"])


files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]
# print(links[0])
# print(files_text[0])
# get_html(links[0])

# # 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # # 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)

# for link in links:
  # get_html(link)

# output_name=soup.find("h1").getText()
output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title 笔趣阁(p2wt)
#@markdown 還在修正的程式，可以直接從這一個區塊執行

def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese


try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  reg=requests.get(art_url)
  reg.encoding="utf-8"
  soup=BeautifulSoup(reg.text)
  content=soup.find(name="div",id='chaptercontent')
  print(art_id)

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)



def toEpub():
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
  if file2epub:
    mdfiles=[ itm for itm in files_text]
    os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
    from google.colab import files
    files.download('../{}.epub'.format(title))
    pass
  else:
    with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
      for file in files_text[::-1]:
        with open(file,"r") as f2:
          f.write(f2.read())
    from google.colab import files
    files.download('../{}.txt'.format(output_name))


#@markdown 書籍目錄網址
url="https://m.bqg9527.net/book/480/" #@param {type:'string'}
title="\u7D66\u4E0D\u8D77\u5F69\u79AE\uFF0C\u53EA\u597D\u5A36\u4E86\u9B54\u9580\u8056\u5973" #@param {type:"string"}
author="\u5149\u5F71" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")], 取得 sites

reg=requests.get(url)
# soup=BeautifulSoup(reg.text,"html.parser")
reg.encoding="utf-8"
soup=BeautifulSoup(reg.text)
output_name=soup.find("h1").getText()
articles=soup.find(name="div",class_="listmain").find_all("a")


links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  if href[-4:]!= "html":
    continue
  links.append([i.text,f"{sites}{href}"])

files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]


# 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)
for link in links:
  get_html(link)


toEpub()


In [ ]:
#@title 愛下電子書(撰寫中)


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")



def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]

  soup=BeautifulSoup(requests.get(art_url).text)
  # content=soup.find(name="div",id='nr1')
  content=soup.find("section")

  print(art_id)

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=re.sub('<script.*</script>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  # text=chinese.s2t(text)

  # print(text)


  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)





#@markdown 書籍目錄網址
url="https://ixdzs8.tw/read/381070/p484.html" #@param {type:'string'}
title="豪門梟士" #@param {type:"string"}
author="雲山風海" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

# reg=requests.get(url)
soup=BeautifulSoup(data)
# articles=soup.find(name="div",id="readerlists").find_all("a")
articles=soup.find_all("a")
# print(articles)


links=[]

for i in articles:
  href=i.get("href")
  itmTitle=i.getText()
  links.append([itmTitle,f"{sites}{href}"])
# links.sort(key=lambda x: x[1])


files_text=[link[1][link[1].rfind("/")+1:-5]+".txt" for link in links]
# links[:4]

# 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)

for link in links:
  get_html(link)

output_name=title
listfiles=os.listdir()
mdfiles=[ itm for itm in files_text if itm in listfiles]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  # mdfiles=[ itm for itm in files_text if itm in listfiles]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
    files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in mdfiles:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title 52書庫(www.52shuku.vip)
#@markdown 還在修正的程式，可以直接從這一個區塊執行

def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese


try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  # print(art_id)
  # print(art_url)
  reg=requests.get(art_url)
  reg.encoding="utf-8"
  soup=BeautifulSoup(reg.text)
  content=soup.find(name="div",id='text')
  print(art_id)

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  # print(text)
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://www.52shuku.vip/yanqing/b/bjPGg.html" #@param {type:'string'}
title="\u59D1\u5A18\u5979\u7F8E\u8C8C\u5374\u66B4\u529B" #@param {type:"string"}
author="\u4E60\u6829\u5112\u751F" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
reg.encoding="utf-8"
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)
output_name=soup.find("h1").getText()
articles=soup.find(name="ul",class_="list").find_all("a")


links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  if href[-4:]!= "html":
    continue
  links.append([i.text,f"{href}"])

files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]



# 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(get_html, links)
for link in links:
  get_html(link)

output_name=soup.find("h1").getText()
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title))
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name))



In [ ]:
#@title 稷下書院(部分文字特別編碼，導致會有亂碼)
#@markdown


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:]


  soup=BeautifulSoup(requests.get(art_url).text)
  print(art_id)
  content=soup.find("div",class_="content")
  advernum=content.find_all("div")
  for itm in advernum:
    itm.clear()


  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"

  # print(text)


  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)





#@markdown 書籍目錄網址
url="https://8book.com/novelbooks/198099/" #@param {type:'string'}
title="煉氣練了三千年" #@param {type:"string"}
author="精裝激光雕刻機" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
reg.encoding = 'UTF-8'
soup=BeautifulSoup(reg.text)
# articles=soup.find(name="div",class_="chaplist").find("dl").find_next("dl").find_all("a")
articles=soup.find("ul", class_="row").find_all('a')


links=[]
for i in articles:
  href=i.get("href")
  itmTitle=i.text
  links.append([itmTitle,f"{sites}{href}"])
# links.sort(key=lambda x: x[1])

files_text=[link[1][link[1].rfind("/")+1:]+".txt" for link in links]

# # 暫時無法使用，目前 colab 只要開放 2 個執行緖、只能運作 60 秒
# # # 同時建立及啟用10個執行緒
# # with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
# #     executor.map(get_html, links)
for link in links:
  get_html(link)

# output_name=title

# if file2epub:
#   mdfiles=[ itm for itm in files_text ]
#   os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
#   from google.colab import files
#   files.download('../{}.epub'.format(title))
#   pass
# else:
#   with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
#     for file in files_text[::-1]:
#       with open(file,"r") as f2:
#         f.write(f2.read())
#   from google.colab import files
#   files.download('../{}.txt'.format(output_name))



目錄已存在


In [ ]:
links[:4]

[['1.煉氣期老祖', 'https://8book.com/read/198099/?12144591'],
 ['2.煉氣練了三千年', 'https://8book.com/read/198099/?12144592'],
 ['3.年年都是大限將至', 'https://8book.com/read/198099/?12144593'],
 ['4.今朝風日好', 'https://8book.com/read/198099/?12144594']]

# 測試後，暫時沒有使用的程式碼區塊

In [ ]:
#@title 檢查 os 錯誤內容


command = "pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles))
try:
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print("Error:", result.stderr)
    else:
        print("Command executed successfully:", result.stdout)
except Exception as e:
    print("An error occurred:", str(e))


Command executed successfully: 


# 效率

透過下述的方法，合併檔案，因為輸出檔需要被反覆的開始太多次，隨著檔案大小逐漸增加。讓效能下跌

```python
for file in files:
  os.system("cat {}>> ../{}.txt".format(file,output_name))
```
若改用下述的方法， output 檔，只需要開啟一次。可以大大縮短時間。

```python
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())
```

# 參考資料

In [ ]:
#@title 多執行序參考程式範例

from bs4 import BeautifulSoup
import concurrent.futures
import requests
import time


def scrape(urls):

    response = requests.get(urls)

    soup = BeautifulSoup(response.content, "lxml")

    # 爬取文章標題
    titles = soup.find_all("h3", {"class": "post_title"})

    for title in titles:
        print(title.getText().strip())

    time.sleep(2)


base_url = "https://www.inside.com.tw/tag/AI"
urls = [f"{base_url}?page={page}" for page in range(1, 6)]  # 1~5頁的網址清單
print(urls)
start_time = time.time()  # 開始時間
# scrape(urls)


# 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     executor.map(scrape, urls)

end_time = time.time()
print(f"{end_time - start_time} 秒爬取 {len(urls)} 頁的文章")